In [1]:
import pandas as pd
import numpy as np
from itertools import chain
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [2]:
from sensor_analysis_functions import fault_detection, plotting_anem_parameter_tunning, anemometer_identification, plot_sensors, get_index_list, split_kneighbor_indexes, generate_dashboard

In [3]:
df_D161075_2016 = pd.read_csv('../Cleaner/data/D161075_2016_data.csv', index_col=0)

In [4]:
df_D161075_2016.sort_values('Timestamp', inplace=True)
df_D161075_2016.reset_index(drop=True, inplace=True)

In [6]:
df_D161075_redundant_sensors = {
    'Ch1Anem': 'Ch2Anem',
    'Ch2Anem': 'Ch1Anem',
    'Ch3Anem': 'Ch4Anem',
    'Ch4Anem': 'Ch3Anem',
    'Ch5Anem': 'Ch6Anem',
    'Ch6Anem': 'Ch5Anem',
}

In [5]:
# Slice de las primeras 7 columnas para no agarrar las columnas con los min y std
df_D161075_2016_fault_detection = fault_detection(df_D161075_2016.iloc[:, :7], sensor='anem', correlation_window=3, ratio_th = 10, correlation_th= .5)

In [7]:
df_D161075_2016_anemometer_identification = anemometer_identification(df_D161075_2016_fault_detection, df_D161075_redundant_sensors)

100%|██████████| 2057/2057 [00:05<00:00, 411.16it/s]


In [8]:
df_D161075_2016_anemometer_identification.name = 'D161075_2016'
generate_dashboard(df_D161075_2016_anemometer_identification, redundance_dict=df_D161075_redundant_sensors)

100%|██████████| 9/9 [00:00<00:00, 329.12it/s]
0it [00:00, ?it/s]
100%|██████████| 5/5 [00:00<00:00, 327.90it/s]
0it [00:00, ?it/s]
100%|██████████| 4/4 [00:00<00:00, 504.23it/s]
0it [00:00, ?it/s]


In [3]:
df_D161075_2017 = pd.read_csv('../Cleaner/data/D161075_2017_data.csv', index_col=0)

df_D161075_2017.sort_values('Timestamp', inplace=True)
df_D161075_2017.reset_index(drop=True, inplace=True)

In [4]:
df_D161075_2017.shape

(52560, 28)

In [12]:
# Slice de las primeras 7 columnas para no agarrar las columnas con los min y std
df_D161075_2017_fault_detection = fault_detection(df_D161075_2017.iloc[:, :7], sensor='anem', correlation_window=3, ratio_th = 10, correlation_th= .5)

KeyboardInterrupt: 

In [10]:
df_D161075_2017_anemometer_identification = anemometer_identification(df_D161075_2017_fault_detection, df_D161075_redundant_sensors)

KeyboardInterrupt: 

In [ ]:
print('yikes')

In [ ]:
df_vaquerias_redundant_sensors = {
    'Ch1Anem': 'Ch2Anem',
    'Ch2Anem': 'Ch1Anem',
    'Ch3Anem': 'Ch2Anem',
    'Ch4Anem': 'Ch5Anem',
    'Ch5Anem': 'Ch4Anem',
    'Ch6Anem': 'Ch5Anem',
}

In [ ]:
df_vaquerias_fault_detection = fault_detection(df_vaquerias, sensor='anem', correlation_window=10, ratio_th = 10, correlation_th= .7)
df_vaquerias_anemometer_identification = anemometer_identification(df_vaquerias_fault_detection, df_vaquerias_redundant_sensors)

In [ ]:
df_vaquerias_anemometer_identification.name = 'Vaquerias'
generate_dashboard(df_vaquerias_anemometer_identification, redundance_dict=df_vaquerias_redundant_sensors)

In [ ]:
def anem_parameter_tunning(df, ch_anem, true_positive_list, ratio_range=30, correlation_range=10, correlation_window_range=10):
    
    df_to_tunning = df[df.columns[df.columns.str.contains('Anem')]].copy()

    idx = 0
    
    for ratio in range(2, ratio_range):
        for correlation in range(1, correlation_range):
            
            correlation = round(correlation *.1, 2)
            
            for window in range(1, correlation_window_range):
                
                df_fault_detection = fault_detection(df_to_tunning, sensor='anem', correlation_window=window, ratio_th=ratio, correlation_th=correlation)

                df_anem_filtered = anemometer_identification(df_fault_detection)

                indexes = get_index_list(df_anem_filtered, ch_anem)

                indexes_splitted = split_kneighbor_indexes(indexes, 12)
                
                
                n_true_positive = 0
                for index in indexes_splitted:
                    for true_positive in true_positive_list:
                        if any(i in true_positive for i in index):
                            n_true_positive += 1
                
                if n_true_positive >= len(true_positive_list) - 1:
                    print(f"Correlation: {correlation}\nCorrelation window: {window}\nRatio: {ratio}\nDetections: {len(split_kneighbor_indexes(indexes, 12))}\nTrue Positives: {n_true_positive}")

In [ ]:
indexes = split_kneighbor_indexes(get_index_list(df_D161075_anemometer_identification, 'Ch3Anem'), 12)

In [ ]:
print(indexes[3][-1])

In [ ]:
#df_D161075_fault_detection.loc[indexes[0][0]-10: indexes[0][0], 'Ch1Anem']
idx = 3

df_D161075_fault_detection.loc[indexes[idx][0]: indexes[idx][-1], df_D161075_fault_detection.columns[df_D161075_fault_detection.columns.str.contains('Ch3Anem')]]#['Ch1Anem','Ch2Anem', 'Timestamp']]#.plot()

In [ ]:
df_D161075_fault_detection.loc[indexes[idx][0] - 10: indexes[idx][-1] + 10, ['Ch3Anem', 'Ch4Anem']].plot(figsize=(25,14))

In [ ]:
np.corrcoef(df_D161075_fault_detection.loc[indexes[idx][0]-3: indexes[idx][0], 'Ch1Anem'].values, df_D161075_fault_detection.loc[indexes[idx][0]-3: indexes[idx][0], 'Ch2Anem'].values)

In [ ]:
ch1_anom = df_D161075_fault_detection.loc[indexes[idx][0] - 3:indexes[idx][0], 'Ch1Anem'].copy()
ch2_anom = df_D161075_fault_detection.loc[indexes[idx][0] - 3:indexes[idx][0], 'Ch2Anem'].copy()

In [ ]:
np.corrcoef(ch1_anom, ch2_anom)

In [ ]:
cols_ch1 = df_D161075_fault_detection.columns[df_D161075_fault_detection.columns.str.contains('Ch1Anem')]
df_D161075_fault_detection.loc[indexes[idx][0]: indexes[idx][0], cols_ch1]

In [ ]:
df_D161075_fault_detection.loc[indexes[idx][0]-3: indexes[idx][0], ['Ch1Anem', 'Ch2Anem', 'Ch3Anem', 'Ch4Anem', 'Ch5Anem', 'Ch6Anem']].plot(figsize=(20,15))

In [ ]:
true_positive_list = [indexes[1], indexes[6], indexes[7], indexes[9], indexes[10], indexes[13], indexes[18]]

In [ ]:
len(true_positive_list)

In [ ]:
anem_parameter_tunning(df_D161075, 'Ch1Anem', true_positive_list, ratio_range=9, correlation_range=11, correlation_window_range=11)